In [1]:
include("functions.jl")

genetique (generic function with 1 method)

# 1) Heuristique gloutonne

L'algorithme glouton choisi:
 - assigne successivement une couleur à chaque nœud
 - les nœuds sont traités dans l'ordre de degré décroissant
 - en choisissant pour chaque nœud la couleur minimisant le nombre de conflits avec ses voisins.

In [2]:
# Comparaison d'une solution aléatoire et d'une solution gloutonne
begin
    instance = instance_list[3]
    aleatoire = sol_alea(instance)

    println(instance.name,"\tk=",instance.k)
    println("SOLUTION ALÉATOIRE:")
    println(aleatoire.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,aleatoire))
    gloutonne = glouton(instance)
    println("\nSOLUTION GLOUTONNE:")
    println(gloutonne.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,gloutonne))
end

dsjc125.1.col	k=5
SOLUTION ALÉATOIRE:


[4, 4, 5, 3, 5, 1, 5, 1, 1, 1, 2, 2, 5, 4, 5, 4, 4, 2, 1, 3, 5, 5, 5, 1, 3, 1, 3, 3, 1, 4, 5, 5, 3, 3, 2, 1, 5, 5, 4, 2, 4, 4, 2, 2, 4, 2, 4, 2, 3, 2, 5, 1, 4, 4, 1, 5, 3, 2, 1, 4, 4, 4, 4, 4, 3, 5, 1, 2, 1, 4, 5, 3, 2, 1, 2, 3, 3, 5, 3, 3, 2, 5, 2, 4, 2, 2, 1, 3, 2, 2, 3, 1, 2, 4, 1, 3, 3, 2, 4, 3, 4, 1, 4, 3, 4, 1, 4, 1, 5, 2, 1, 2, 4, 4, 5, 4, 4, 4, 1, 2, 3, 2, 3, 2, 1]
collisions: 136



SOLUTION GLOUTONNE:
[3, 2, 2, 4, 2, 5, 4, 3, 4, 3, 2, 5, 3, 3, 2, 4, 4, 4, 3, 5, 5, 5, 5, 4, 5, 2, 3, 3, 2, 5, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 5, 4, 2, 2, 5, 4, 3, 5, 3, 2, 2, 2, 4, 3, 4, 4, 4, 5, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 5, 4, 4, 4, 3, 4, 2, 2, 2, 5, 3, 3, 3, 2, 4, 3, 3, 5, 5, 4, 2, 5, 3, 2, 3, 4, 5, 2, 5, 5, 3, 4, 4, 3, 2, 2, 5, 2, 4, 5, 5, 2, 4, 4, 3, 2, 2, 3]
collisions: 47


## Tests numériques de l'heuristique gloutonne

In [3]:
begin
    nsamples = 10
    println("INSTANCE NAME AND k\tMIN CONFL\tMEAN CONFL\tMAX CONFL\tTOTAL TIME\tTIME BEST SOL\tSOLS PER SECOND")
    for instance ∈ instance_list
        conflicts_samples = Vector{Int}(undef,nsamples)
        time_samples = zeros(nsamples)
        for i = 1:nsamples
            solution = glouton(instance)
            start_time = time_ns()
            conflicts = nbr_collision(instance,solution)
            run_time = time_ns()-start_time
            conflicts_samples[i] = conflicts
            time_samples[i] = run_time*1e-9
        end
        println(
            instance.name," k=",instance.k,"\t",
            minimum(conflicts_samples),"\t",
            sum(conflicts_samples)/nsamples,"\t",
            maximum(conflicts_samples),"\t",
            sum(time_samples),"\t",
            time_samples[argmin(conflicts_samples)],"\t",
            nsamples/sum(time_samples)
        ) # solutions per second irrelevant, isn't it? (since there is no local search loop)
    end
end

INSTANCE NAME AND k	MIN CONFL	MEAN CONFL	MAX CONFL	TOTAL TIME	TIME BEST SOL	SOLS PER SECOND


flat300_26_0.col k=26	226	226.0	226	0.0021532	0.00021270000000000002	4644.250417982537
le450_15c.col k=15	362	362.0	362	0.0025183000000000002	0.00025600000000000004	3970.9327721081677
dsjc125.1.col k=5	47	47.0	47	8.81e-5	1.3100000000000002e-5	113507.37797956867
dsjc125.9.col k=44	21	21.0	21	0.0001424	1.69e-5	70224.7191011236
dsjc250.1.col k=8	88	88.0	88	0.0005706000000000001	0.0001341	17525.411847178406
dsjc250.9.col k=72	61	61.0	61	0.0007152	6.27e-5	13982.102908277404
dsjc250.5.col k=28	85	85.0	85	0.0015517	0.00014240000000000002	6444.544692917445


dsjc1000.5.col k=86	378	378.0	378	0.024249899999999998	0.0026517000000000003	412.37283452715275


dsjc1000.5.col k=85	398	398.0	398	0.023473800000000003	0.0021720000000000003	426.0068672306997


dsjc1000.5.col k=84	412	412.0	412	0.024034	0.0025705000000000003	416.0772239327619


# 2) Structure de voisinage

On choisit d'utiliser un opérateur de voisinage simple:
 - Appliquer la couleur $i$ au nœud $j$ choisis de manière à réduire le plus possible le nombre de conflits par rapport à la solution courante.
   - On choisit le premier couple $(i,j)$ occasionnant la plus forte réduction de conflits.
   - Pour cela, on maintient une matrice des conflits stockant pour chaque nœud $j$ et chaque couleur $i$ le nombre de conflits en ce nœud $j$ si l'on lui applique la couleur $i$.

On obtient donc une structure de voisinage qui contient toutes les solutions dont un unique nœud diffère par sa couleur de la solution d'origine.

In [7]:
# Test de descente locale simple
begin
    instance = instance_list[3]
    solution = glouton(instance)

    for t = 1:25
        simple_neighbor(instance,solution)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 43
t: 2	conflits: 40
t: 3	conflits: 38
t: 4	conflits: 36
t: 5	conflits: 34
t: 6	conflits: 32
t: 7	conflits: 30
t: 8	conflits: 28
t: 9	conflits: 27
t: 10	conflits: 26
t: 11	conflits: 25
t: 12	conflits: 24
t: 13	conflits: 23
t: 14	conflits: 22
t: 15	conflits: 21
t: 16	conflits: 20
t: 17	conflits: 19
t: 18	conflits: 18
t: 19	conflits: 17
t: 20	conflits: 16
t: 21	conflits: 15
t: 22	conflits: 15
t: 23	conflits: 15
t: 24	conflits: 15
t: 25	conflits: 15


# 3) Métaheuristique proposée

On choisit d'écrire un

Tabou

In [5]:
# Test de descente locale avec tabou
begin
    instance = instance_list[3]
    solution = sol_alea(instance)

    for t = 1:500
        tabu_neighbor(instance,solution,10,t)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 143
t: 2	conflits: 138
t: 3	conflits: 133
t: 4	conflits: 128
t: 5	conflits: 124
t: 6	conflits: 120
t: 7	conflits: 116
t: 8	conflits: 112
t: 9	conflits: 108
t: 10	conflits: 105
t: 11	conflits: 102
t: 12	conflits: 99
t: 13	conflits: 96
t: 14	conflits: 93
t: 15	conflits: 90
t: 16	conflits: 87
t: 17	conflits: 84
t: 18	conflits: 81
t: 19	conflits: 78
t: 20	conflits: 76
t: 21	conflits: 74
t: 22	conflits: 71
t: 23	conflits: 69
t: 24	conflits: 67
t: 25	conflits: 65
t: 26	conflits: 63
t: 27	conflits: 61
t: 28	conflits: 59
t: 29	conflits: 57
t: 30	conflits: 55
t: 31	conflits: 53
t: 32	conflits: 52
t: 33	conflits: 51
t: 34	conflits: 50
t: 35	conflits: 48
t: 36	conflits: 47
t: 37	conflits: 46
t: 38	conflits: 45
t: 39	conflits: 44
t: 40	conflits: 43
t: 41	conflits: 42
t: 42	conflits: 41
t: 43	conflits: 40
t: 44	conflits: 39
t: 45	conflits: 38
t: 46	conflits: 37
t: 47	conflits: 36
t: 48	conflits: 35
t: 49	conflits: 34
t: 50	conflits: 33
t: 51	conflits: 32
t: 52	conflits: 32
t: 53	conf

Algo génétique

In [6]:
begin
    # pass
end